In [ ]:
#Importing Libraries
import os
import tensorflow as tf
import numpy as np 
import pandas as pd
import statistics
import seaborn as sns
import collections
import IPython
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adamax
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

#Music21 is an object-oriented toolkit for analyzing, 
#searching, and transforming music in symbolic (scorebased) forms
!pip install music21
import music21

In [ ]:
#Loading the list of midi files as stream 
filepath = "../input/lofi-midi/"

#Getting midi files
all_midis = [music21.converter.parse(filepath+i) for i in os.listdir(filepath)]

In [ ]:
#Extraction function        
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = music21.instrument.partitionByInstrument(j) 
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, music21.note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, music21.chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))
    return notes

#Getting the list of notes as Corpus
Corpus = extract_notes(all_midis)
print("Total notes in all the Lofi midis in the dataset:", len(Corpus))

In [ ]:
#First 50 values (notes) in the corpus
print("First fifty values in the Corpus:", Corpus[:50])

In [ ]:
#Function for distinction between notes and chords
def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Seperating the notes in chord
            notes = [] 
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)            
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)
        # pattern is a note
        else: 
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 1
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

In [ ]:
#Counting unique notes in the corpus
count_num = collections.Counter(Corpus)
print("Total unique notes in the Corpus:", len(count_num))

In [ ]:
#Exploring the notes dictionary
Notes = list(count_num.keys())
Recurrence = list(count_num.values())

print("Average recurrenc for a note in Corpus:", round(statistics.mean(Recurrence), 2))
print("Most frequent note in Corpus appeared:", max(Recurrence), "times")
print("Least frequent note in Corpus appeared:", min(Recurrence), "time")

In [ ]:
# Plotting the distribution of Notes
plt.figure(figsize=(18,3),facecolor="#97BACB")
bins = np.arange(0,(max(Recurrence)), 50) 
plt.hist(Recurrence, bins=bins, color="#97BACB")
plt.axvline(x=100,color="#DBACC1")
plt.title("Frequency Distribution Of Notes In The Corpus")
plt.xlabel("Frequency Of Chords in Corpus")
plt.ylabel("Number Of Chords")
plt.show()

In [ ]:
# Storing all the unique characters present in my corpus to build a mapping dict 
symb = sorted(list(set(Corpus)))

L_corpus = len(Corpus) #length of corpus
L_symb = len(symb) #length of total unique characters

#Building dictionary to access the vocabulary from indices and vice versa
mapping = dict((c, i) for i, c in enumerate(symb))
reverse_mapping = dict((i, c) for i, c in enumerate(symb))

print("Total number of characters:", L_corpus)
print("Number of unique characters:", L_symb)

In [ ]:
#Splitting the Corpus in equal length of strings and output target
length = 4
features = []
targets = []
for i in range(0, L_corpus - length, 1):
    feature = Corpus[i:i + length]
    target = Corpus[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])
    
L_datapoints = len(targets)
print("Total number of sequences in the Corpus:", L_datapoints)

In [ ]:
# reshape X and normalize
X = (np.reshape(features, (L_datapoints, length, 1)))/ float(L_symb)
# encode the output variable
y = tf.keras.utils.to_categorical(targets) 

In [ ]:
#Taking out a subset of data to be used as seed
X_train, X_seed, y_train, y_seed = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Initialising the Model
model = Sequential()

#Adding layers
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))

model.add(LSTM(256))

model.add(Dense(256))
model.add(Dropout(0.1))

model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
#Compiling the model for training  
opt = Adamax(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt)

In [ ]:
#Model's Summary               
model.summary()

In [ ]:
#Training the Model
cb = [tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience=50, 
                                              restore_best_weights=True),
             tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss' , patience=50)]

history = model.fit(X_train, y_train, batch_size=256, epochs=200 , callbacks=cb)

In [ ]:
#Plotting the learning curve for the loss function 
history_df = pd.DataFrame(history.history)
fig = plt.figure(figsize=(15,4), facecolor="#97BACB")
fig.suptitle("Learning Plot of Model for Loss")
pl=sns.lineplot(data=history_df["loss"],color="#444160")
pl.set(ylabel ="Training Loss")
pl.set(xlabel ="Epochs")

**Génération de la mélodie:**

In [ ]:
from music21 import *
#function to obtain the generated music
def Malody_Generator(Note_Count):
    seed = X_seed[np.random.randint(0,len(X_seed)-1)]
    Music = ""
    Notes_Generated=[]
    for i in range(Note_Count):
        seed = seed.reshape(1,length,1)
        prediction = model.predict(seed, verbose=0)[0]
        prediction = np.log(prediction) / 1.0 #diversity
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        index = np.argmax(prediction)
        index_N = index/ float(L_symb)   
        Notes_Generated.append(index)
        Music = [reverse_mapping[char] for char in Notes_Generated]
        seed = np.insert(seed[0],len(seed[0]),index_N)
        seed = seed[1:]
    #Now, we have music in form or a list of chords and notes and we want to be a midi file.
    Melody = chords_n_notes(Music)
    Melody_midi = stream.Stream(Melody)   
    return Music,Melody_midi


#getting the Notes and Melody created by the model
Music_notes, Melody = Malody_Generator(100)

In [ ]:
#To save the generated melody
Melody.write('midi','Melody_Generated5.mid')

In [ ]:
#to play audio or corpus
IPython.display.Audio(r"/kaggle/input/generated-midis/Melody_Generated1.wav")

In [ ]:
#to play audio or corpus
IPython.display.Audio("/kaggle/input/generated-midis/Melody_Generated2.wav")

In [ ]:
#to play audio or corpus
IPython.display.Audio(r"/kaggle/input/generated-midis/Melody_Generated3.wav")

In [ ]:
#to play audio or corpus
IPython.display.Audio(r"/kaggle/input/generated-midis/Melody_Generated4.wav")

In [ ]:
#to play audio or corpus
IPython.display.Audio(r"/kaggle/input/generated-midis/Melody_Generated5.wav")